# Dynet segmentation with tf fold
![animation](../../fold/tensorflow_fold/g3doc/animation.gif)  

In [1]:
#just a bunch of fun
import numpy as np
import six
from multiprocessing import Process, Queue
import time
import data
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)
import tensorflow_fold as td
from conv_lstm_cell import *

# params
EMBEDDING_SIZE = 64
SEP = "|"
data_dir = "/home/moon/data/"

#our alphabet

vocabulary = data.vocabulary(data_dir + 'vocabulary')
vsize=len(vocabulary)
print(vocabulary)

index = lambda char: vocabulary.index(char)
char = lambda i: vocabulary[i]



[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'C', 'D', 'E', 'G', 'L', 'N', 'S', 'T', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '²', '³', '¹', 'á', 'é', 'í', 'ó', 'ö', 'ú', 'ü', 'ő', 'ű']


In [2]:
class reader():
    def __init__(self, folder, qsize=10000, start=True):
        self.data_dir = folder
        self.qsize= 10000
        self.pool = dict()
        self.data = dict()
        self.datasets = ["train", "test", "validation"]
        for dataset in self.datasets:
            self.data[dataset] = Queue(qsize)
            self.pool[dataset] = Process(target=self.sentence_reader, args=(self.data[dataset], self.data_dir+dataset))
        if start == True:
            self.start()
            
    def sentence_reader(self, queue, file):
        """
        read sentences from the data format setence: word\tword\n.....\t\n
        """
        while True:
            with open(file) as f:
                while True:
                    try:
                        sentence = []
                        while True:
                            line = f.readline()[:-1].split('\t')
                            if line[0] != "":
                                sentence.append(line)
                            else:
                                break
                        sent = " ".join([word[0] for word in sentence])
                        segmented = " ".join([word[1].replace(" ","|") for word in sentence])
                        tags = []
                        last_char = "_"
                        for char in segmented:
                            if char != "|":
                                tags.append(0 if last_char!="|" else 1)
                            last_char = char
                        queue.put((sent, tags))
                    except e:
                        print(e)
                        
    def start(self):
        for dataset in self.datasets:
            self.pool[dataset].start()
            
    def stop(self):
        for dataset in self.datasets:
            self.pool[dataset].terminate()
            
    def get(self,dataset):
        if dataset in self.datasets:
            return self.data[dataset].get()
        else:
            raise KeyError
            
store = reader("/home/moon/data/", start=False)


## helper functions

In [3]:
def params_info():
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        print(variable.name, shape)
        # print(len(shape))
        variable_parametes = 1
        for dim in shape:
            # print(dim)
            variable_parametes *= dim.value
        print("\tparams: ", variable_parametes)
        total_parameters += variable_parametes
    print(total_parameters)
    return total_parameters

def onehot(string):
    onehot = np.zeros([len(string),vsize])
    onehot[np.arange(len(string)), np.array([index(char) for char in string])]=1
    return [onehot[i,:] for i in range(len(onehot))]
onehot("kacsa")

[array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  

In [4]:
cell = td.ScopedLayer(tf.contrib.rnn.BasicLSTMCell(num_units=16), 'char_cell')
convlstm = Conv1DLSTMCell(input_shape=[vsize,1], output_channels=8, kernel_shape=[5])
conv_lstm_cell_1d = td.ScopedLayer(convlstm)


In [770]:
def net():
    comp = td.Composition()
    with comp.scope():
        forward = td.Identity().reads(comp.input[0])
        backward = td.Identity().reads(comp.input[0])
        """
        model1 = (td.InputTransform(lambda s: [index(x) for x in s]) >>
                 td.Map(td.Scalar(tf.int32) >>
                 td.Function(lambda indices: tf.one_hot(indices, depth=vsize)) >>
                 td.Function(lambda x: tf.reshape(x, [-1,vsize,1]))) >>             #onehot encoding
                 td.RNN(conv_lstm_cell_1d)  >> td.GetItem(1) >> td.GetItem(0) ).reads(comp.input)
        model2 = (td.InputTransform(lambda s: [index(x) for x in s]) >>
                 td.Map(td.Scalar(tf.int32) >>
                 td.Function(lambda indices: tf.one_hot(indices, depth=vsize)) >>
                 td.Function(lambda x: tf.reshape(x, [-1,vsize,1]))) >>             #onehot encoding
                 td.RNN(conv_lstm_cell_1d) >> td.GetItem(1) >> td.GetItem(0) ).reads(comp.input)

        rnn_outs = td.Concat().reads(model1, model2)
        out = td.Function(lambda rnn_outs: tf.contrib.layers.flatten(rnn_outs)).reads(rnn_outs)
        #fc = out >> td.FC(300) >> td.FC(1)
        """
        forw = (td.Map(td.Vector(vsize) >> 
                td.Function(lambda x: tf.reshape(x, [-1,vsize,1]))) >>
                td.RNN(conv_lstm_cell_1d) >> td.GetItem(1) >> td.GetItem(0) >>
                td.Function(lambda rnn_outs: tf.contrib.layers.flatten(rnn_outs))).reads(forward)
        backw = (td.Map(td.Vector(vsize) >> 
                td.Function(lambda x: tf.reshape(x, [-1,vsize,1]))) >>
                td.RNN(conv_lstm_cell_1d) >> td.GetItem(1) >> td.GetItem(0) >>
                td.Function(lambda rnn_outs: tf.contrib.layers.flatten(rnn_outs))).reads(backward)
        rnn_outs = td.Concat().reads(forw, backw)
        comp.output.reads(rnn_outs)

    full = comp >> td.FC(300) >>td.FC(200) >> td.FC(30) >> td.FC(1, activation=tf.nn.sigmoid)
    
    cell = td.ScopedLayer(tf.contrib.rnn.BasicLSTMCell(num_units=16), 'char_cell')
    #forward = td.Map(td.Vector(vsize) >> td.Function(lambda x: tf.reshape(x, [-1,vsize]))) >> td.RNN(cell) >> td.GetItem(1) >> td.GetItem(0) >> td.FC(1)

    #conv = td.Map(td.Vector(vsize) >> td.Function(lambda x: tf.reshape(x, [-1,vsize,1]))) >> td.RNN(conv_lstm_cell_1d) >> td.GetItem(1) >> td.GetItem(0) >> td.Function(lambda rnn_outs: tf.contrib.layers.flatten(rnn_outs)) >> td.FC(1)
    
    return full

In [771]:
#print(forward.eval(onehot("kacsa")).shape)
nn = net()
compiler = td.Compiler.create((nn,td.Scalar()))
model_output, target = compiler.output_tensors
loss = tf.nn.l2_loss(model_output - target)
train_op = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)#tf.train.AdamOptimizer(0.001).minimize(loss)


/home/moon/Envs/tf1/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [779]:
nn = net()
a=tuple([onehot("ser"),onehot("vami")])
print(a)
print(nn.eval(a))
#print(new.input_type, new.output_type)
#sess.run(tf.global_variables_initializer())
#compiler.loom_input_tensor
#sess.run(tf.local_variables_initializer())
#sess.run(train_op,{compiler.loom_input_tensor: ["aa","bb"]})

([array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,

TypeError: Constant is not of a recognized TypeShape: TypeShape(dtype='float32', shape=(3, 60), tag='')

In [740]:
#sess.run(tf.global_variables_initializer())

sess.run([loss, train_op], compiler.build_feed_dict([(onehot("nysdafae"), 1) for _ in range(3)]))


[1.0971727, None]

In [741]:
params_info()

char_cell_2/kernel:0 (17, 64)
	params:  1088
char_cell_2/bias:0 (64,)
	params:  64
FC_1/bias:0 (1,)
	params:  1
FC_1/weights:0 (16, 1)
	params:  16
char_cell_4/kernel:0 (17, 64)
	params:  1088
char_cell_4/bias:0 (64,)
	params:  64
FC_1_2/bias:0 (1,)
	params:  1
FC_1_2/weights:0 (16, 1)
	params:  16
char_cell_5/kernel:0 (17, 64)
	params:  1088
char_cell_5/bias:0 (64,)
	params:  64
FC_1_3/bias:0 (1,)
	params:  1
FC_1_3/weights:0 (16, 1)
	params:  16
char_cell_6/kernel:0 (18, 64)
	params:  1152
char_cell_6/bias:0 (64,)
	params:  64
FC_1_4/bias:0 (1,)
	params:  1
FC_1_4/weights:0 (16, 1)
	params:  16
char_cell_7/kernel:0 (76, 64)
	params:  4864
char_cell_7/bias:0 (64,)
	params:  64
FC_1_5/bias:0 (1,)
	params:  1
FC_1_5/weights:0 (16, 1)
	params:  16
char_cell_8/kernel:0 (76, 64)
	params:  4864
char_cell_8/bias:0 (64,)
	params:  64
FC_1_6/bias:0 (1,)
	params:  1
FC_1_6/weights:0 (16, 1)
	params:  16
char_cell_9/kernel:0 (76, 64)
	params:  4864
char_cell_9/bias:0 (64,)
	params:  64
FC_1_7/bi

1101008